In [1]:
import glob
import pandas as pd
import numpy as np

In [91]:
def is_empty(e):
    if e is None:
        return True
    if isinstance(e, str) and not e:
        return True
    if isinstance(e, float) and np.isnan(e):
        return True
    if hasattr(e, '__len__') and len(e) == 0:
        return True
    return False

def is_string(e):
    if isinstance(e, str) and e:
        return True
    return False

def is_number(e):
    if isinstance(e, float) and not np.isnan(e):
        return True
    if isinstance(e, str) and e:
        return (e
                .replace(" ", "")
                .replace("-", "")
                .replace("%", "")
                .replace(",", "")
                .isdigit()
               )
    return False

def identification_mask(pdf):
    def substitute(e):
        if is_empty(e):
            return None
        if is_number(e):
            return "n"
        if is_string(e):
            return "s"
    return pdf.applymap(substitute)

In [92]:
ic = identification_mask(c)

In [93]:
ic

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,None,None,None,None,None,None,None,None,None,None,...,s,None,None,None,None,None,None,None,None,s
1,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,s
2,None,None,n,None,n,None,n,None,n,None,...,n,n,None,n,None,n,None,n,s,s
3,s,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,n,n
4,s,None,n,None,n,None,n,None,n,None,...,n,n,None,n,None,n,None,n,None,n
5,s,None,n,None,n,None,n,None,n,None,...,n,n,None,n,None,n,None,None,None,n
6,s,None,n,None,n,None,n,None,n,None,...,n,n,None,n,None,None,None,None,None,n
7,s,None,n,None,n,None,n,None,n,None,...,n,n,None,None,None,None,None,None,None,n
8,s,None,n,None,n,None,n,None,n,None,...,n,None,None,None,None,None,None,None,None,n
9,s,None,n,None,n,None,n,None,n,None,...,None,None,None,None,None,None,None,None,None,n


In [94]:
def identify_number_area(c):
    ic = identification_mask(c)
    r1, r2 = iter(range(c.shape[0]-1, -1, -1)), iter(range(c.shape[1]-1, -1, -1))

    i, j = 1, 1
    ie, je = -1, -1

    while True:
        if ie == -1:
            i = next(iter(r1))
        if je == -1:
            j = next(iter(r2))
        
        if (ic.iloc[i, j+1:] == 's').any():
            ie = i + 1
        if i == 0 and ie == -1:
            ie = 0
        
        if (ic.iloc[i+1:, j] == 's').any():
            je = j + 1
        if j == 0 and je == -1:
            je = 0
        
        if ie != -1 and ie != -1:
            return ie, je

In [201]:
def squeeze_headers(c, number_area):
    
    def apply_transform(s):
        return s.apply(str).apply(lambda x: "" if x == "NaN" else x)
    
    sc = c.applymap(lambda x: "NaN" if isinstance(x, float) and np.isnan(x) else x)
    ie, je = number_area
    
    l1 = None
    for i in range(0, ie):
        if l1 is None:
            l1 = apply_transform(sc.iloc[0, :])
        else:
            l1 = l1.str.cat(apply_transform(sc.iloc[i]), sep=' ')
    if l1 is not None:
        sc = pd.concat([l1.to_frame().T, sc.drop(axis=0, index=range(0, ie))])
    
    l2 = None
    for i in range(0, je):
        if l2 is None:
            l2 = apply_transform(sc.iloc[:, 0])
        else:
            l2 = l2.str.cat(apply_transform(sc.iloc[:, i]), sep=' ')
    if l2 is not None:
        sc = pd.concat([l2.to_frame(), sc.drop(axis=1, columns=range(0, je))], axis=1)
    
    if l1 is not None:
        sc.iloc[0, :] = [f"*{i}" if x.strip() == "" else x for i, x in enumerate(sc.iloc[0, :])]
        sc.columns = sc.iloc[0, :]
        sc = sc.drop(axis=0, index=[0])
        
    sc = sc.set_index(sc.columns[0])
    
    return sc

def squeeze_columns(sc):
    columns_to_drop = []
    for c1, c2 in zip(sc.columns[1:], sc.columns[2:]):
        all_is_nan = (sc[c1] == "NaN").all()
        if not isinstance(all_is_nan, pd.Series) and all_is_nan and c2[0] == "*":
            sc[c1] = sc[c2]
            columns_to_drop.append(c2)
            
    return sc.drop(columns=columns_to_drop)

In [205]:
total_numbers = 0
for i, f in enumerate(glob.glob("../data/res/*/*.csv", recursive=True)):
    print("parsing", i, f)
    c = pd.read_csv(f, sep=',', thousands=' ', header=None, dtype=object, encoding='UTF-8')
    display(c)
    ic = identification_mask(c)
#     display(ic)
    na = identify_number_area(c)
    print(na)
    sc = squeeze_headers(c, na)
#     display(sc)
    display(squeeze_columns(sc))
    total_numbers += (ic.iloc[na[0]:, na[1]:] == "n").count().sum()
    
print("total_numbers", total_numbers)

parsing 0 ../data/res\generali\tabula-Generali_2019_EN_short-0.csv


,0,1
0,NaN,Solvency II value
1,Assets,NaN
2,Intangible assets,0
3,Deferred tax assets,0
4,Pension benefit surplus,0
5,"Property, plant & equipment held for own use",877 909
6,Investments (other than assets held for index-...,68 036 488
7,Property (other than for own use),316 964
8,"Holdings in related undertakings, including pa...",17 794 170
9,Equities,2 086 266


(1, 1)


,Solvency II value
*0,
Assets,NaN
Intangible assets,0
Deferred tax assets,0
Pension benefit surplus,0
"Property, plant & equipment held for own use",877 909
Investments (other than assets held for index-linked and unit-linked contracts),68 036 488
Property (other than for own use),316 964
"Holdings in related undertakings, including participations",17 794 170
Equities,2 086 266


parsing 1 ../data/res\generali\tabula-Generali_2019_EN_short-1.csv


...

parsing 21 ../data/res\generali\tabula-Generali_2019_EN_short-9.csv


,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Development year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Year end
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(discounted
2,NaN,NaN,0,NaN,1,NaN,2,NaN,3,NaN,...,5,6,NaN,7,NaN,8,NaN,9,10 & +,data)
3,Prior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,420 364,377 027
4,N-9,NaN,0,NaN,0,NaN,0,NaN,0,NaN,...,0,0,NaN,0,NaN,0,NaN,162 994,NaN,146 406
5,N-8,NaN,0,NaN,0,NaN,0,NaN,0,NaN,...,0,0,NaN,0,NaN,164 871,NaN,NaN,NaN,147 961
6,N-7,NaN,0,NaN,0,NaN,0,NaN,0,NaN,...,0,0,NaN,144 318,NaN,NaN,NaN,NaN,NaN,129 259
7,N-6,NaN,0,NaN,0,NaN,0,NaN,0,NaN,...,0,198 013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178 342
8,N-5,NaN,0,NaN,0,NaN,0,NaN,0,NaN,...,322 650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,288 757
9,N-4,NaN,0,NaN,0,NaN,0,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,413 781


(3, -1)


,*1,0,*3,1,*5,2,*7,3,*9,4,...,Development year 5,6,*14,7,*16,8,*18,9,10 & +,Year end (discounted data)
*0,,,,,,,,,,,,,,,,,,,,,
Prior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,420 364,377 027
N-9,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,...,0,0,NaN,0,NaN,0,NaN,162 994,NaN,146 406
N-8,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,...,0,0,NaN,0,NaN,164 871,NaN,NaN,NaN,147 961
N-7,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,...,0,0,NaN,144 318,NaN,NaN,NaN,NaN,NaN,129 259
N-6,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,...,0,198 013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178 342
N-5,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,...,322 650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,288 757
N-4,NaN,0,NaN,0,NaN,0,NaN,0,NaN,457 880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,413 781
N-3,NaN,0,NaN,0,NaN,0,NaN,702 378,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,641 280
N-2,NaN,0,NaN,0,NaN,1 487 727,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 386 980


total_numbers 2230
